In [1]:
# Exploring map matching stats

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1609092362676_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# spark = sparkSession
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

# installing required packages for this notebook session
sc.install_pypi_package("boto3")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/55/c6/912cc2cfd1b4051621552fc5961c25e2f517a090d179a38f62d5cdaf5d37/boto3-1.16.43-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ef/6c/9f6e6a14f53b21b6f1670ccd789082015911458823914b7dabca333ae033/botocore-1.19.43-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f5/71/45d36a8df68f3ebb098d6861b2c017f3d094538c0fb98fa61d4dc43e69b9/urllib3-1.26.2-py2.py3-none-any.whl

In [3]:
# Getting stats about distance between lat,long of the trace and the nearest shape

from pyspark.sql import functions as F
import boto3

# new_line header
csv_out = "day,shape_distance_mean,shape_distance_min,shape_distance_max,shape_distance_stddev,shape_distance_quantile_25,shape_distance_quantile_50,shape_distance_quantile_75,total_size,shape_distance_higher_1000m,shape_distance_higher_4000m\n"


# from october 1 to october 31 
for day in range(1,32):
    
    # reading files
    traces_shape_distance = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/8-map-matching/MO_1510" + str(day) + "/")    
    
    total = traces_shape_distance.count()
    higher_1000 = traces_shape_distance.filter("min_distance > 1000").count()
    higher_4000 = traces_shape_distance.filter("min_distance > 4000").count()
    
    # getting shape_min_distance info (mean,min,max,stddev)
    stats = traces_shape_distance.agg(F.mean('min_distance').alias('mean'),
                       F.min('min_distance').alias('min'),
                       F.max('min_distance').alias('max'),
                       F.stddev('min_distance').alias("stddev")).collect()

    # getting shape_min_distance quantiles (25%, 50%, 75%)
    # 0.0001 is the precision
    shape_distance_quantile = traces_shape_distance.approxQuantile("min_distance", [0.25,0.5,0.75], 0.0001)
    
    
    csv_out += "MO_1510" + str(day) +","+ str(stats[0]["mean"]) +","+ str(stats[0]["min"]) + "," + str(stats[0]["max"]) + "," + str(stats[0]["stddev"]) + "," +str(shape_distance_quantile[0]) + "," +str(shape_distance_quantile[1]) +"," +str(shape_distance_quantile[2]) + "," + str(total) +","+ str(higher_1000) + ","+ str(higher_4000) + "\n"



s3 = boto3.client('s3')

# writing results in S3
s3.put_object(Body=bytes(csv_out,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/exploring-data/5-shape_distance-stats.csv')    


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'E32DC2303A538C49', 'HostId': 'I6/eK8vS9wlt6wyQXCWTJ0IrR1qFG6tARGgXpcz0skdMTEzPvwBIOPcDXSjhLySQCr6PbTTgb4I=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'I6/eK8vS9wlt6wyQXCWTJ0IrR1qFG6tARGgXpcz0skdMTEzPvwBIOPcDXSjhLySQCr6PbTTgb4I=', 'x-amz-request-id': 'E32DC2303A538C49', 'date': 'Sun, 27 Dec 2020 20:39:10 GMT', 'etag': '"d3578d76555551336210ebfdab6ca82e"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"d3578d76555551336210ebfdab6ca82e"'}

In [4]:
traces_shape_distance = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/8-map-matching/MO_1510" + str(day) + "/")    
traces_shape_distance.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+----------+----------+--------+------------------+---------------+--------------+---------+---------+--------+---------------+--------+-----------------+------------------+-------------------+-------------------+
|              dt_avl|id_avl|line_id|   trace_y|   trace_x|hour_avl|         hour_diff|         region|time_variation|  trip_id|direction|route_id|      trip_head|shape_id|     min_distance|min_shape_sequence|min_shape_coord_lat|min_shape_coord_lon|
+--------------------+------+-------+----------+----------+--------+------------------+---------------+--------------+---------+---------+--------+---------------+--------+-----------------+------------------+-------------------+-------------------+
|2015-10-29 06:11:...|  8565|  33045|-23.572727|-46.448793|       6|1.5800000429153442|PARQUE DO CARMO|          45.0|3024-10-1|        1| 3024-10|Vila Industrial|   59555|9069.537797648747|             202.0|-23.608742000000003|         -46.528649|


In [ ]:
# Results
- Distance mean during work days - 436m and 466m
- Distance mean for weekend days - 1007m and 1396m
- Min value - 0 min
- Max value - 54km
- 25% quantile - 9 e 18
- 50% quantile - 20 e 38, some value higher than 180 at sundays (4,11,18,25), and in a holiday october 12
- 75% quantile - 53 e 57, some value higher than 1390m at weekends
- around 2mi registers higher than 1000m
- around 600000 register with distance more than 4000m